# T1. 使用 SDK 工具
`fastie` 除了支持命令行工具，还支持 `Python` `SDK`，可以方便的在 `Python` 脚本中使用 `fastie`.

`fastie` 的 `SDK` 工具主要包括三个模块:

- `dataset` 模块
- `task` 模块
- `controller` 模块

## 1. dataset 模块

`dataset` 即为命令行参数中的 `--dataset`，用于指定数据集. 例如在上一节基础教程 `T0. 使用命令行工具`
中, 我们使用了 `jsonlines-ner` 作为参数来加载数据集, 那么在 `python` 中, 我们可以使用如下代码来加载数据集:

```python
from fastie.dataset.io import JsonLinesNER

data_bundle = JsonLinesNER(folder="/path/to/dataset").run()
```

注意 `fastie` 中的模块都有传入参数和 `run` 两步, 传入参数用于指定模块的参数, `run` 用于执行模块.

`fastie` 中 `dataset` 获得的数据集是 `fastNLP.io.DataBundle` 类型, 即包含多个子集, 例如
`train`, `dev`, `test`, `infer`, 详细的使用方法可以参考
[fastNLP 文档](https://fastnlp.readthedocs.io/zh/latest/)

不同的 `controller` 会使用不同的子集, 例如 `train` 控制器会使用 `train` 子集和 `dev` 子集
(如果要在训练阶段同时进行验证), `eval` 控制器会使用 `test` 子集, `infer` 控制器会使用 `infer` 子集.

## 2. task 模块

`task` 即为命令行参数中的 `--task`，用于指定任务. 例如在上一节基础教程 `T0. 使用命令行工具`
中, 我们使用了 `ner/bert` 作为参数来加载任务, 那么在 `python` 中, 我们可以使用如下代码来加载任务:

```python
from fastie.tasks.ner import BertNER

task = BertNER(batch_size=32, load_model="path/to/model").run(data_bundle)
```

与 `dataset` 类似, `task` 也有传入参数和 `run` 两步, 传入参数用于指定模块的参数, `run` 用于执行模块,
不同的是, `task` 的 `run` 方法需要传入 `data_bundle` 参数, 也就是 `dataset`, `task`,
`controll` 具有顺序关系, 体现在每一步的 `run` 要传入上一步的结构.

## 3. controller 模块

`controller` 即为命令行模式中的可执行部分, 例如 `fastie-train`，用于指定控制器. 例如在上一节基础教程
`T0. 使用命令行工具`, 我们使用了 `fastie-train` 作为启动来加载控制器, 那么在 `python` 中,
我们可以使用如下代码来加载控制器:

```python
from fastie.controller import Trainer

state_dict = Train().run(task)
```

与 `dataset` 和 `task` 类似, `controller` 也有传入参数和 `run` 两步, 传入参数用于指定模块的参数,
`run` 用于执行模块, 不同的是, `controller` 的 `run` 方法需要传入 `task` 参数, 也就是 `dataset`,
`task`, `controll` 具有顺序关系, 体现在每一步的 `run` 要传入上一步的结构.

`controller` 的 `run` 返回结果与控制器的类型有关, 当使用 `train` 控制器时, `run` 返回的
结果是 `task` 的 `state_dict`, 包含模型参数; 当使用 `eval` 控制器时, `run` 返回的结果是
`json` 的验证结果; 当使用 `infer` 控制器时, `run` 返回的结果是的推理结果.

## 4. 使用 controller 模块 加载配置

前文 `T0. 使用命令行工具` 提到了使用 `config` 文件可以方便地对 `task`, `dataset` 以及其他
参数进行储存, 在 `SDK` 方式中也可以使用 `config` 文件, 例如:

```python
from fastie.controller import Trainer

state_dict = Trainer.from_config("config.py").run()
```

如上所示, 使用 `controller` 的类方法 `from_config`, 即可不指定 `controller` 对象 `run`
方法的参数, 具体的 `task` 和 `dataset` 将会到配置文件中搜索并自动运行.

`controller` 的 `from_config` 除了支持文件类型的 `config` 外, 还可以输入 `dict` 类型的
`config`, 例如:

```python
from fastie.controller import Trainer

config = {
    "task": "ner/bert",
    "dataset": "conll2003",
    "batch_size": 32,
    "device": [0, 1]
}
state_dict = Trainer.from_config(config).run()
```

上述的使用与配置文件基本一致, 只是将配置文件的内容写道了代码里.

## 5. 技巧: 重复使用 task 和 dataset

当您使用 `controller` 训练完成后, 可直接使用 `task` 的结果进行验证和推理, 例如:

```python
from fastie.controller import Trainer, Evaluation, Inference
from fastie.tasks.ner import BertNER
from fastie.dataset.io import Conll2003

data_bundle = Conll2003().run()
task = BertNER(batch_size=32, load_best_model=True).run(data_bundle)

Trainer().run(task)
Evaluation().run(task)
Inference().run(task)
```

如上所示, 当 `Trainer().run(task)` 执行结束后, `task` 中的 `model` 就已经训练完成,
因此可以直接使用同一个 `task` 结果进行验证和推理, 当然, 如果要进行验证, 请确保您的 `data_bundle`
中包含 `test` 集; 如果要进行推理, 请确保您的 `data_bundle` 中包含 `infer` 集.